# Algoritmo XGBoost

## Tratamento da base de dados

In [2]:
import pandas as pd
import numpy as np
base_casas = pd.read_csv('house_prices.csv')
base_casas

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [3]:
base_casas.drop(columns=['id', 'date', 'sqft_living15', 'sqft_lot15'], axis=1, inplace=True)
base_casas

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069


In [4]:
base_treinamento = base_casas.iloc[0:15129,:]
base_treinamento.shape

(15129, 17)

In [5]:
base_teste = base_casas.iloc[15129:,:]
base_teste.shape

(6484, 17)

In [6]:
X_teste = base_teste.iloc[:,1:17].values
X_teste

array([[ 4.00000e+00,  2.75000e+00,  2.58000e+03, ...,  9.81150e+04,
         4.76753e+01, -1.22304e+02],
       [ 4.00000e+00,  2.50000e+00,  3.20000e+03, ...,  9.80590e+04,
         4.75273e+01, -1.22143e+02],
       [ 3.00000e+00,  1.00000e+00,  8.40000e+02, ...,  9.81780e+04,
         4.74940e+01, -1.22275e+02],
       ...,
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75944e+01, -1.22299e+02],
       [ 3.00000e+00,  2.50000e+00,  1.60000e+03, ...,  9.80270e+04,
         4.75345e+01, -1.22069e+02],
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75941e+01, -1.22299e+02]])

In [7]:
y_teste = base_teste.iloc[:,0].values
y_teste

array([937750., 725126., 135000., ..., 402101., 400000., 325000.])

In [8]:
base_treinamento.to_csv('house_prices_train_xgboost.csv', header=False, index=False)
base_teste.to_csv('house_prices_test_xgboost.csv', header=False, index=False)

## Configurações SageMaker

In [9]:
import sagemaker
import boto3
from sagemaker import Session

In [10]:
session = sagemaker.Session()
bucket = 'praticaawssagemaker'
subpasta_modelo = 'modelos/house-prices/xgboost'
subpasta_dataset = 'datasets/house-prices'
key_train = 'houses-train-data-xgboost'
key_test = 'houses-test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = f's3://{bucket}/{subpasta_dataset}/train/{key_train}'
s3_test_data = f's3://{bucket}/{subpasta_dataset}/test/{key_test}'
output_location = f's3://{bucket}/{subpasta_modelo}'
print(f'Role: {role}')
print(f'Localização da base de treinamento: {s3_train_data}')
print(f'Localização da base de teste: {s3_test_data}')
print(f'Modelo final será salvo em: {output_location}')

Role: arn:aws:iam::804019055874:role/service-role/AmazonSageMaker-ExecutionRole-20230719T151197
Localização da base de treinamento: s3://praticaawssagemaker/datasets/house-prices/train/houses-train-data-xgboost
Localização da base de teste: s3://praticaawssagemaker/datasets/house-prices/test/houses-test-data-xgboost
Modelo final será salvo em: s3://praticaawssagemaker/modelos/house-prices/xgboost


In [11]:
import os
with open('house_prices_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [12]:
import os
with open('house_prices_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

## Treinamento do XGBoost

In [13]:
from sagemaker import image_uris
container = image_uris.retrieve(framework='xgboost', region=boto3.Session().region_name, version='latest')

In [14]:
xgboost = sagemaker.estimator.Estimator(
    image_uri=container, role=role, instance_count=1,
    instance_type='ml.m5.2xlarge', output_path=output_location, sagemaker_session=session
)

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [15]:
xgboost.set_hyperparameters(num_round=100)

In [16]:
train_input = sagemaker.inputs.TrainingInput(s3_data=s3_train_data, content_type='csv', s3_data_type='S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data=s3_test_data, content_type='csv', s3_data_type='S3Prefix')
data_channels = {'train':train_input, 'validation':validation_input}

In [17]:
xgboost.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-2023-10-18-18-36-17-581


2023-10-18 18:36:17 Starting - Starting the training job...
2023-10-18 18:36:37 Starting - Preparing the instances for training......
2023-10-18 18:37:39 Downloading - Downloading input data...
2023-10-18 18:38:14 Training - Downloading the training image...
2023-10-18 18:38:24 Training - Training image download completed. Training in progress.Arguments: train
[2023-10-18:18:38:43:INFO] Running standalone xgboost training.
[2023-10-18:18:38:43:INFO] File size need to be processed in the node: 1.55mb. Available memory size in the node: 23968.9mb
[2023-10-18:18:38:43:INFO] Determined delimiter of CSV input is ','
[18:38:43] S3DistributionType set as FullyReplicated
[18:38:43] 15129x16 matrix with 242064 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-10-18:18:38:43:INFO] Determined delimiter of CSV input is ','
[18:38:43] S3DistributionType set as FullyReplicated
[18:38:43] 6484x16 matrix with 103744 entries loaded from /opt/ml/input/data/validati

## Deploy, Previsões e Avaliação 

In [18]:
xgboost_regressor = xgboost.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-10-18-18-40-31-009
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-10-18-18-40-31-009
INFO:sagemaker:Creating endpoint with name xgboost-2023-10-18-18-40-31-009


-----!

In [19]:
from sagemaker.serializers import CSVSerializer
xgboost_regressor.serializer = CSVSerializer()

In [21]:
previsoes = np.array(xgboost_regressor.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)
previsoes

array([715760.25 , 759194.25 , 125879.664, ..., 298663.34 , 468468.56 ,
       292532.9  ], dtype=float32)

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)
print(f"""
    MAE = {mae}
    MSE = {mse}
""")


    MAE = 69311.54741358536
    MSE = 14874845482.575417



## Tuning